1. Loading & pre-processing models

The models are available at:
•iML1515: http://bigg.ucsd.edu/models/iML1515
•eciML1515_batch: https://github.com/SysBioChalmers/ecModels/blob/chore/updateiML1515/eciML1515/model


In [1]:
from cobra.io import read_sbml_model

In [2]:
import cameo
met_model = cameo.load_model("iML1515")

In [3]:
import cobra
! [ ! -f "eciML1515_batch.xml" ] && curl -O -L "https://raw.githubusercontent.com/SysBioChalmers/ecModels/chore/updateiML1515/eciML1515/model/eciML1515_batch.xml"
ec_model = cobra.io.read_sbml_model("eciML1515_batch.xml")


In [4]:
import math

In [5]:
# Replacing any infinite bound by 1000 in the ecModel:
for reaction in ec_model.reactions:
    if math.isinf(reaction.upper_bound):
        reaction.upper_bound = 1000

# Remove repetition in ecModel met ids:
for metabolite in ec_model.metabolites:
    metabolite.id = metabolite.id.split("[")[0]

# Close biotin uptake:EX_ser__L_e
met_model.reactions.EX_ser__L_e.lower_bound = 0
ec_model.reactions.EX_ser__L_e_REV.upper_bound = 0

2. Simulating the model

In [6]:
def simulate_ec_model(model, rxn_id):
    # First optimization: maximize input
    model.objective = rxn_id
    solution = model.optimize()
    
    # Second optimization: minimize glucose
    model.reactions.get_by_id(rxn_id).lower_bound = solution.fluxes[rxn_id]
    glc_rxn = model.reactions.get_by_id("EX_glc__D_e_REV")
    model.objective = {glc_rxn: -1}
    solution = cobra.flux_analysis.pfba(model)
    
    return solution

      


In [7]:
def print_fluxes(model, solution, bio_rxn_id):
    # Show growth:
    print("growth: " + str(solution.fluxes[bio_rxn_id]))
    
    # Show exchange reactions:
    for rxn in model.reactions:
        if len(rxn.metabolites) == 1 and solution.fluxes[rxn.id] != 0:
            print(rxn.id + ": " + str(solution.fluxes[rxn.id]))   

2.1 Wild type conditions:

In [8]:
ec_model.objective.expression.args[0]

1.0*BIOMASS_Ec_iML1515_core_75p37M

In [9]:
# Metabolic model:
biomass_rxn_id = "BIOMASS_Ec_iML1515_core_75p37M"
met_sol_biomass = cobra.flux_analysis.pfba(met_model)
print_fluxes(met_model, met_sol_biomass, biomass_rxn_id)

growth: 0.8769972144269691
EX_pi_e: -0.8459567750194346
EX_co2_e: 24.00329327297647
EX_h_e: 8.058200328043306
EX_mn2_e: -0.0006060050751690357
EX_fe2_e: -0.01408545226091155
EX_glc__D_e: -10.0
EX_zn2_e: -0.0002990560501195965
DM_amob_c: 1.7539944288539381e-06
EX_mg2_e: -0.007607950835153957
EX_ca2_e: -0.004564770501092374
EX_ni2_e: -0.000283270100259911
EX_meoh_e: 1.7539944288539381e-06
EX_cu2_e: -0.0006217910250287211
EX_cobalt2_e: -2.1924930360674228e-05
EX_h2o_e: 47.16236480869412
EX_mobd_e: -6.138980500988784e-06
EX_so4_e: -0.22084543853699973
EX_nh4_e: -9.471495371047805
EX_k_e: -0.17118371727564338
EX_cl_e: -0.004564770501092374
DM_5drib_c: 0.00019732437324606806
EX_o2_e: -22.131763238946302
DM_4crsol_c: 0.00019557037881721411


In [10]:
# ecModel:
ec_sol_biomass = simulate_ec_model(ec_model, biomass_rxn_id)
print_fluxes(ec_model, ec_sol_biomass, biomass_rxn_id)


growth: 0.5743654341118459
EX_h_e: 134.23853676145438
EX_co2_e: 0.44365060832324055
EX_pyr_e: 128.96132171755437
EX_glyclt_e: 0.0002561669836139424
EX_lac__D_e: -3.311129148642067e-12
EX_h2o_e: 143.61591832922525
EX_etoh_e: 0.9548031058528785
DM_5drib_c: 0.00025616698361388327
DM_mththf_c: 0.00025616698361388327
EX_pi_e_REV: 0.5540363439368159
EX_mn2_e_REV: 0.00039688651497128555
EX_btn_e_REV: 1.1487308682236917e-06
EX_fe2_e_REV: 0.009224883237270356
EX_glc__D_e_REV: 68.80204146625192
EX_thm_e_REV: 0.00012808349180694163
EX_zn2_e_REV: 0.00019585861303213946
EX_mg2_e_REV: 0.004982620140920263
EX_ca2_e_REV: 0.0029895720845521585
EX_ni2_e_REV: 0.00018552003521812624
EX_cobalt2_e_REV: 1.435913585279615e-05
EX_nh4_e_REV: 6.202577492262535
EX_mobd_e_REV: 4.020558038782922e-06
EX_so4_e_REV: 0.14450632266450206
EX_k_e_REV: 0.11211211218059354
EX_o2_e_REV: 62.74441833679438
EX_cu_e_REV: 0.00040722509278529875
EX_cl_e_REV: 0.0029895720845521585
prot_pool_exchange: 0.0909113178074544


2.2 Increased L-serine conditions:

In [11]:
# Metabolic model:
Serine_rxn_id = "EX_ser__L_e"
met_model.reactions.get_by_id(biomass_rxn_id).lower_bound = 0.8 * met_sol_biomass.fluxes[biomass_rxn_id]
met_model.objective = Serine_rxn_id
met_sol_serine = cobra.flux_analysis.pfba(met_model)
print_fluxes(met_model, met_sol_serine, biomass_rxn_id)

growth: 0.7015977715415753
EX_pi_e: -0.6767654200157409
EX_co2_e: 18.288266299990973
EX_h_e: 10.751349701898004
EX_mn2_e: -0.00048480406013522853
EX_fe2_e: -0.01126836180872924
EX_glc__D_e: -10.0
EX_ser__L_e: 4.304789439463108
EX_zn2_e: -0.0002392448400956772
DM_amob_c: 1.4031955430831507e-06
EX_mg2_e: -0.006086360668123166
EX_ca2_e: -0.0036518164008739
EX_ni2_e: -0.00022661608020792883
EX_meoh_e: 1.4031955430831507e-06
EX_cu2_e: -0.000497432820022977
EX_cobalt2_e: -1.7539944288539383e-05
EX_h2o_e: 41.12031296802939
EX_mobd_e: -4.911184400791027e-06
EX_so4_e: -0.17667635082959982
EX_nh4_e: -11.881985736301777
EX_k_e: -0.1369469738205147
EX_cl_e: -0.0036518164008739
DM_5drib_c: 0.00015785949859685447
EX_o2_e: -18.943436992498604
DM_4crsol_c: 0.0001564563030537713


In [12]:
# ecModel:
ec_model.reactions.get_by_id(biomass_rxn_id).lower_bound = 0.8 * ec_sol_biomass.fluxes[biomass_rxn_id]
ec_sol_serine = simulate_ec_model(ec_model, Serine_rxn_id)
print_fluxes(ec_model, ec_sol_serine, biomass_rxn_id)

growth: 0.45949234728947674
EX_h_e: 119.39706529211908
EX_ac_e: 2.6989523382512175
EX_pyr_e: 110.9493171445139
EX_ser__L_e: 1.5270205659256448
EX_tre_e: -9.126424060923455e-08
EX_glyclt_e: 0.00020493358689122054
EX_lac__D_e: 3.2083057064368153e-06
EX_h2o_e: 122.31790554898666
EX_etoh_e: 2.2910313575495893
DM_5drib_c: 0.00020493358689110662
DM_mththf_c: 0.00020493358689110548
EX_pi_e_REV: 0.44322907514957244
EX_mn2_e_REV: 0.00031750921197702843
EX_btn_e_REV: 9.189846945789534e-07
EX_fe2_e_REV: 0.007379906589816285
EX_glc__D_e_REV: 61.04483508542177
EX_thm_e_REV: 0.00010246679344555331
EX_zn2_e_REV: 0.00015668689042571155
EX_mg2_e_REV: 0.003986096112736211
EX_ca2_e_REV: 0.002391657667641727
EX_ni2_e_REV: 0.000148416028174501
EX_cobalt2_e_REV: 1.1487308682236919e-05
EX_nh4_e_REV: 6.489082559735857
EX_mobd_e_REV: 3.216446431026337e-06
EX_so4_e_REV: 0.11560505813160146
EX_k_e_REV: 0.08968968974447483
EX_o2_e_REV: 52.96706547810774
EX_cu_e_REV: 0.000325780074228239
EX_cl_e_REV: 0.00239165766

In [13]:
import pandas as pd

usage_df = pd.DataFrame()
for reaction in ec_model.reactions:
    if "draw_prot_" in reaction.id:
        gene_id = reaction.gene_reaction_rule
        new_line = pd.DataFrame(index = [reaction.id[-6:]],
                                data = {"gene":[met_model.genes.get_by_id(gene_id).name],
                                        "biomass_usage":[ec_sol_biomass.fluxes[reaction.id]],
                                        "serine_usage":[ec_sol_serine.fluxes[reaction.id]]})
        usage_df = usage_df.append(new_line)

print(usage_df)


        gene  biomass_usage  serine_usage
O32583  thiS   0.000000e+00  0.000000e+00
P00350   gnd   0.000000e+00  0.000000e+00
P00363  frdA   0.000000e+00  0.000000e+00
P00370  gdhA   4.731638e-08  5.265849e-08
P00393   ndh   0.000000e+00  0.000000e+00
...      ...            ...           ...
Q59385  copA   0.000000e+00  0.000000e+00
Q6BEX0  ytfR   0.000000e+00  0.000000e+00
Q6BF16  dgoA   0.000000e+00  0.000000e+00
Q6BF17  dgoD   0.000000e+00  0.000000e+00
Q93K97  nudF   0.000000e+00  0.000000e+00

[1259 rows x 3 columns]


Let's make sure all values are positive:


In [14]:
usage_df = usage_df.sort_values(by=['biomass_usage'])
print(usage_df.head(n=5))
usage_df = usage_df.sort_values(by=['serine_usage'])
print(usage_df.head(n=5))

        gene  biomass_usage  serine_usage
O32583  thiS            0.0           0.0
P32674  pflD            0.0           0.0
P32669  fsaB            0.0           0.0
P32664  nudC            0.0           0.0
P32662   gph            0.0           0.0
        gene  biomass_usage  serine_usage
O32583  thiS            0.0           0.0
P0AAI1  ssuB            0.0           0.0
P0ABK5  cysK            0.0           0.0
P0ABK9  nrfA            0.0           0.0
P0ABL1  nrfB            0.0           0.0


One value was slightly negative, probably due to solver approximations. Let's change that:

In [15]:
usage_df.loc[usage_df["serine_usage"] < 0, "serine_usage"] = 0
print(usage_df.head(n=5))

        gene  biomass_usage  serine_usage
O32583  thiS            0.0           0.0
P0AAI1  ssuB            0.0           0.0
P0ABK5  cysK            0.0           0.0
P0ABK9  nrfA            0.0           0.0
P0ABL1  nrfB            0.0           0.0


There are a lot of rows with zero usage under both conditions, so let's filter them out:

In [16]:
usage_df = usage_df.query("biomass_usage > 0 or serine_usage > 0")
print(usage_df)

        gene  biomass_usage  serine_usage
P77580  mhpF   3.722318e-09  0.000000e+00
P52647   pfo   1.845907e-07  0.000000e+00
P13482  treA   0.000000e+00  1.273958e-13
P76536  yfeX   1.368444e-12  1.094755e-12
P52643  ldhA   0.000000e+00  1.220824e-12
...      ...            ...           ...
P0A825  glyA   1.962638e-05  1.011990e-04
P0A953  fabB   1.552053e-04  1.241642e-04
P00934  thrC   8.802668e-05  1.764638e-04
P21151  fadA   2.255986e-04  1.804788e-04
P0A9B2  gapA   2.695271e-04  2.394442e-04

[294 rows x 3 columns]


Now let's compute usage changes. We will look at both absolute changes (the difference between both conditions) and relative changes (the fold change or ratio between them).

In [17]:
usage_df["abs_changes"] = usage_df["serine_usage"] - usage_df["biomass_usage"]
usage_df["rel_changes"] = usage_df["serine_usage"] / usage_df["biomass_usage"]
print(usage_df)

        gene  biomass_usage  serine_usage   abs_changes  rel_changes
P77580  mhpF   3.722318e-09  0.000000e+00 -3.722318e-09     0.000000
P52647   pfo   1.845907e-07  0.000000e+00 -1.845907e-07     0.000000
P13482  treA   0.000000e+00  1.273958e-13  1.273958e-13          inf
P76536  yfeX   1.368444e-12  1.094755e-12 -2.736888e-13     0.800000
P52643  ldhA   0.000000e+00  1.220824e-12  1.220824e-12          inf
...      ...            ...           ...           ...          ...
P0A825  glyA   1.962638e-05  1.011990e-04  8.157265e-05     5.156275
P0A953  fabB   1.552053e-04  1.241642e-04 -3.104106e-05     0.800000
P00934  thrC   8.802668e-05  1.764638e-04  8.843708e-05     2.004662
P21151  fadA   2.255986e-04  1.804788e-04 -4.511971e-05     0.800000
P0A9B2  gapA   2.695271e-04  2.394442e-04 -3.008292e-05     0.888386

[294 rows x 5 columns]


In [18]:
usage_df = usage_df.sort_values(by=['abs_changes'])
print(usage_df.tail(n=20).iloc[::-1])

        gene  biomass_usage  serine_usage   abs_changes  rel_changes
P00934  thrC   8.802668e-05  1.764638e-04  8.843708e-05     2.004662
P0A825  glyA   1.962638e-05  1.011990e-04  8.157265e-05     5.156275
P0A9N4  pflA   2.208713e-05  5.080758e-05  2.872045e-05     2.300325
P09373  pflB   2.208713e-05  5.080758e-05  2.872045e-05     2.300325
P24186  folD   6.912624e-06  2.898666e-05  2.207404e-05     4.193294
P00547  thrB   1.924166e-05  3.857197e-05  1.933031e-05     2.004608
P00562  metL   1.668142e-05  3.213054e-05  1.544912e-05     1.926128
P08660  lysC   7.686946e-06  1.363058e-05  5.943637e-06     1.773212
P0A9Q9   asd   2.327062e-06  4.126374e-06  1.799312e-06     1.773212
P07003  poxB   2.359086e-06  3.919941e-06  1.560855e-06     1.661636
P00864   ppc   1.365867e-06  1.878193e-06  5.123260e-07     1.375092
P04693  tyrB   1.195994e-06  1.589883e-06  3.938886e-07     1.329340
P39451  adhP   1.611326e-07  3.866345e-07  2.255019e-07     2.399480
P28861   fpr   0.000000e+00  2.669

•  20 of enzymes that decreased their absolute usage the most:

In [19]:
print(usage_df.head(n=20))


        gene  biomass_usage  serine_usage  abs_changes  rel_changes
P21151  fadA       0.000226      0.000180    -0.000045     0.800000
P0A953  fabB       0.000155      0.000124    -0.000031     0.800000
P0A9B2  gapA       0.000270      0.000239    -0.000030     0.888386
P17854  cysH       0.000118      0.000094    -0.000024     0.800000
P0AC62  grxC       0.000118      0.000094    -0.000024     0.800000
P0A9C5  glnA       0.000083      0.000067    -0.000017     0.800000
P0A6E4  argG       0.000069      0.000055    -0.000014     0.800000
P36683  acnB       0.000065      0.000052    -0.000013     0.800000
P0A6C5  argA       0.000061      0.000048    -0.000012     0.800000
P16690  phnN       0.000046      0.000037    -0.000009     0.800000
P0A955   eda       0.000067      0.000059    -0.000007     0.887892
P0A6A8  acpP       0.000033      0.000026    -0.000007     0.800000
P31119   aas       0.000032      0.000026    -0.000006     0.800000
P11446  argC       0.000032      0.000026    -0.

• 20 of enzymes that increased their relative usage the most

In [20]:
usage_df = usage_df.sort_values(by=['rel_changes'])
print(usage_df.tail(n=20).iloc[::-1])

        gene  biomass_usage  serine_usage   abs_changes  rel_changes
P52643  ldhA   0.000000e+00  1.220824e-12  1.220824e-12          inf
P28861   fpr   0.000000e+00  2.669459e-08  2.669459e-08          inf
P13482  treA   0.000000e+00  1.273958e-13  1.273958e-13          inf
P0A825  glyA   1.962638e-05  1.011990e-04  8.157265e-05     5.156275
P24186  folD   6.912624e-06  2.898666e-05  2.207404e-05     4.193294
P39451  adhP   1.611326e-07  3.866345e-07  2.255019e-07     2.399480
P0A9N4  pflA   2.208713e-05  5.080758e-05  2.872045e-05     2.300325
P09373  pflB   2.208713e-05  5.080758e-05  2.872045e-05     2.300325
P00934  thrC   8.802668e-05  1.764638e-04  8.843708e-05     2.004662
P00547  thrB   1.924166e-05  3.857197e-05  1.933031e-05     2.004608
P00562  metL   1.668142e-05  3.213054e-05  1.544912e-05     1.926128
P08660  lysC   7.686946e-06  1.363058e-05  5.943637e-06     1.773212
P0A9Q9   asd   2.327062e-06  4.126374e-06  1.799312e-06     1.773212
P07003  poxB   2.359086e-06  3.919

• 20 of enzymes that decreased their relative usage the most:

In [21]:
print(usage_df.head(n=20))

        gene  biomass_usage  serine_usage   abs_changes  rel_changes
P52647   pfo   1.845907e-07  0.000000e+00 -1.845907e-07     0.000000
P77580  mhpF   3.722318e-09  0.000000e+00 -3.722318e-09     0.000000
P0A6A3  ackA   3.245591e-07  5.773203e-09 -3.187859e-07     0.017788
P77218  eutD   1.173068e-06  1.731961e-07 -9.998716e-07     0.147644
P0ABY4  fldB   2.961680e-06  2.248366e-06 -7.133140e-07     0.759152
P0ADU2  ygiN   4.278245e-07  3.335485e-07 -9.427594e-08     0.779639
P0AGD3  sodB   1.112328e-07  8.672137e-08 -2.451139e-08     0.779639
P0A7B3  nadK   5.705306e-10  4.564245e-10 -1.141061e-10     0.800000
P0ABQ0   dfp   1.592795e-07  1.274236e-07 -3.185591e-08     0.800000
P0A790  panD   1.654367e-07  1.323494e-07 -3.308734e-08     0.800000
P0A6I9  coaE   2.235631e-06  1.788505e-06 -4.471263e-07     0.800000
P0A6I3  coaA   9.432622e-08  7.546098e-08 -1.886524e-08     0.800000
P31663  panC   7.680894e-08  6.144715e-08 -1.536179e-08     0.800000
P0A6I6  coaD   2.215811e-08  1.772

4. Using cameo

We will use the FSEOF approach from cameo on the ecModel to validate the results from the previous section:


In [22]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

def ec_fseof(ec_model, biomass_rxn_id, target_rxn_id):
    # Revert the ecModel to its original state:
    ec_model.reactions.get_by_id(biomass_rxn_id).lower_bound = 0
    ec_model.reactions.get_by_id(target_rxn_id).lower_bound = 0
    ec_model.objective = biomass_rxn_id
    
    # Build a list with all rxns except the "draw_prot_XXXXXX" ones:
    nonDrawList = []
    for reaction in ec_model.reactions:
        if "draw_prot" not in reaction.id:
            nonDrawList.append(reaction)

    # Run the FSEOF analysis, filtering out any reaction that is not a "draw_prot_XXXXXX" rxn:
    fseof = FSEOF(ec_model)
    fseof_result = fseof.run(target=ec_model.reactions.get_by_id(target_rxn_id), exclude=nonDrawList)
    fseof_df = fseof_result.data_frame
    return fseof_df

fseof_df = ec_fseof(ec_model, biomass_rxn_id, Serine_rxn_id)
print(fseof_df.loc[:, "5":])

                        5         6         7         8         9         10
draw_prot_P00547  0.000062  0.000071  0.000079  0.000088  0.000096  0.000105
draw_prot_P00562  0.000051  0.000058  0.000065  0.000071  0.000078  0.000085
draw_prot_P00864  0.000002  0.000003  0.000003  0.000003  0.000003  0.000004
draw_prot_P00934  0.000284  0.000323  0.000362  0.000402  0.000441  0.000480
draw_prot_P02918  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
draw_prot_P04693  0.000002  0.000002  0.000002  0.000003  0.000003  0.000003
draw_prot_P07003  0.000007  0.000008  0.000009  0.000010  0.000010  0.000011
draw_prot_P07623  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
draw_prot_P08660  0.000021  0.000023  0.000026  0.000029  0.000031  0.000034
draw_prot_P09373  0.000086  0.000099  0.000111  0.000124  0.000137  0.000149
draw_prot_P0A6A3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000001
draw_prot_P0A6D3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000

We will replace reaction ids with gene names. First, we have to replace with the associated gene ids:


In [23]:
def rxn_ids2gene_ids(df, ec_model):
    id_dict = {}
    for reaction in ec_model.reactions:
        if "draw_prot_" in reaction.id:
            id_dict[reaction.id] = reaction.gene_reaction_rule
    df = df.rename(mapper=id_dict)
    return df
    
fseof_df = rxn_ids2gene_ids(fseof_df, ec_model)
print(fseof_df.loc[:, "5":].head(n=5))


             5         6         7         8         9         10
b0003  0.000062  0.000071  0.000079  0.000088  0.000096  0.000105
b3940  0.000051  0.000058  0.000065  0.000071  0.000078  0.000085
b3956  0.000002  0.000003  0.000003  0.000003  0.000003  0.000004
b0004  0.000284  0.000323  0.000362  0.000402  0.000441  0.000480
b3396  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000


In [24]:
def gene_ids2gene_names(df, met_model):
    id_dict = {}
    for gene in met_model.genes:
        id_dict[gene.id] = gene.name
    df = df.rename(mapper=id_dict)
    return df

fseof_df = gene_ids2gene_names(fseof_df, met_model)
print(fseof_df.loc[:, "5":].head(n=5))

            5         6         7         8         9         10
thrB  0.000062  0.000071  0.000079  0.000088  0.000096  0.000105
metL  0.000051  0.000058  0.000065  0.000071  0.000078  0.000085
ppc   0.000002  0.000003  0.000003  0.000003  0.000003  0.000004
thrC  0.000284  0.000323  0.000362  0.000402  0.000441  0.000480
mrcA  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000


Finally, we can sort based on the slope of increase in serine production as each protein usage increases:


In [25]:
import numpy as np

def sort_by_slope(df):
    # For each row, create a linear model with the serine exchange as prediction, and store the slope of said model:
    slopes = pd.DataFrame()
    for index, row in df.iterrows():
        if sum(row) == 0:
            slopes.loc[index,"slope"] = 0
        else:
            x = row
            y = df.loc["EX_ser__L_e"]
            A = np.vstack([x, np.ones(len(x))]).T
            m, c = np.linalg.lstsq(A, y, rcond=None)[0]
            slopes.loc[index,"slope"] = m
            
    # Add computed slopes to the dataframe and use them for sorting:
    df["slope"] = slopes
    df = df.sort_values(by=["slope"], ascending=False)
    return df

fseof_df = sort_by_slope(fseof_df)
print(fseof_df.loc[:, "slope":])


                    slope
adhP         3.391660e+06
ackA         3.391660e+06
tyrB         2.826384e+06
ppc          2.620828e+06
asd          8.352596e+05
poxB         7.420813e+05
eutD         3.935232e+05
lysC         2.581454e+05
metL         9.946607e+04
thrB         7.925699e+04
folD         6.938259e+04
pflA         5.339593e+04
pflB         5.339593e+04
glyA         1.872578e+04
thrC         1.731230e+04
EX_ser__L_e  1.000000e+00
gpmA        -2.051745e+06
metH        -2.243714e+06
ispG        -2.243714e+06
fldB        -2.243714e+06
lpxC        -2.713328e+06
proB        -2.713328e+06
glmM        -2.713328e+06
pyrG        -2.713328e+06
cysN        -2.826384e+06
cysD        -2.826384e+06
aroL        -2.826384e+06
metA        -3.391660e+06
pyrD        -3.391660e+06
murA        -3.391660e+06
folP        -3.391660e+06
purC        -3.391660e+06
cysI        -3.391660e+06
cysJ        -3.391660e+06
hemL        -3.391660e+06
lpxA        -3.391660e+06
thiL        -3.391660e+06
katE        

Overall we see consensus with the genes from section 3: 
    in particular, all genes that cameo predicted to increase in expression as L-serine prediction increases (top 15 genes in the list) 
    are also in the top 20 genes (expect genes: ackA, eutD, and folD) that increase in terms of relative usage. This validates 
    1) cameo as a tool for exploring ecModels, and 
    2) the genes that we had proposed originally.
